In [3]:
! pip install --upgrade google-cloud-bigquery
! pip install --upgrade google-cloud-storage
! pip install --upgrage google-cloud
! pip install pyarrow

     |████████████████████████████████| 188 kB 907 kB/s eta 0:00:01
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 2.11.0
    Uninstalling google-cloud-bigquery-2.11.0:
      Successfully uninstalled google-cloud-bigquery-2.11.0
     |████████████████████████████████| 103 kB 898 kB/s eta 0:00:01


  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.36.2
    Uninstalling google-cloud-storage-1.36.2:
      Successfully uninstalled google-cloud-storage-1.36.2

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --upgrage


In [15]:
from google.cloud import bigquery
import pyarrow
import pandas as pd

GOOGLE_SERVICE_ACCOUNT_FILE = '/Users/mzc01-yunsu/Documents/yunsu_servant.json'
client = bigquery.Client.from_service_account_json(GOOGLE_SERVICE_ACCOUNT_FILE)

In [16]:
# Construct a BigQuery client object.

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "utopian-surface-268707.YS_DATASET.COMMENTS"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, 
    skip_leading_rows=1, 
    autodetect=True,
    ignore_unknown_values=True,
    allow_quoted_newlines=True,
    allow_jagged_rows=True
)

with open("/Users/mzc01-yunsu/Downloads/preprocessing_nbs.csv", "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.

print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows,
        len(table.schema),
        table_id
        )
    )

Loaded 598 rows and 15 columns to utopian-surface-268707.YS_DATASET.COMMENTS


In [24]:
sql = '''
WITH
  B AS (
  WITH
    A AS (
    SELECT
      *
    FROM
      `utopian-surface-268707.YS_DATASET.COMMENTS2` )
  SELECT
    * EXCEPT(title,
      comments,
      description,
      tags),
    REGEXP_REPLACE(tags, r"[^가-힣A-Za-z0-9]", " ") AS tags,
    REGEXP_REPLACE(title, r"[^가-힣A-Za-z0-9]", " ") AS title,
    REGEXP_REPLACE(description, r"[^가-힣A-Za-z0-9]", " ") AS description,
    REGEXP_REPLACE(comments, r"[\\n]", " ") AS comments
  FROM
    A )
SELECT
  * EXCEPT(comments),
  REGEXP_REPLACE(comments, r"[^가-힣A-Za-z0-9_ ]", "") AS comments
FROM
  B
'''
job = client.query(sql)
df = job.result().to_dataframe()
df.head()

,published_at,published_time,published_year,published_month,published_day,month_upload_count,duration,view_count,likes,dislikes,comment_count,tags,title,description,comments
0,2016-10-30,15,2016,10,30,13,411,3642,51.0,1.0,38.0,단체사진 가슴운동 벤치프레스 vlog 복학생 가슴 루틴 가슴운동 루틴 간단 운동 짧...,단체사진 도전 가슴운동 벤치프레스 vlog 복학생 Ep 28,유도부 단체사진을 찍어봤어요 밖에서 찍는 건 처음이라 시간이 꽤 오래 걸렸네...,같이운동하면서 배워가고싶네여 멋지십니다 아이구 덕분에 학교 오랜만에 보네요 참 추억...
1,2016-10-26,15,2016,10,26,13,571,7163,58.0,2.0,69.0,스쿼트 어깨 대학생활 중간고사 vlog 복학생 밀리터리프레스 레터럴레이즈 레그프레스...,중간고사 끝 스쿼트 어깨 vlog 대학생활 복학생 Ep27,중간고사가 드디어 끝났습니다 사실은 과목이 하나 더 남았어요 그런데 아직 일주일 ...,스트랩 손가락에 끼는거 이름이뭐에요정보좀주세요 치의예과 지렸다 클라스 헐 존경스러울...
2,2016-10-23,11,2016,10,23,13,783,14422,64.0,2.0,59.0,경돼 힘든만큼 등운동 유도훈련 일상 복학생 vlog 데드리프트 덤벨컬 해머컬 유도대...,힘든만큼 등 운동 유도 훈련 일상 vlog 복학생 Ep26,힘든만큼 성장할 겁니다 분명 오늘은 전반적인 등 운동과 유도 훈련을 했어...,엉터리 생고기n오랜만에 보네요 경돼님 개인적인 생각으론 파워리프팅은 취미로 할 수 ...
3,2016-10-22,4,2016,10,22,13,576,5692,46.0,3.0,39.0,살 찌는 가슴운동 벤치프레스 일상 vlog 복학생 경돼 기본 운동 3대 운동 루틴 ...,살 찌는 중 가슴운동 벤치프레스 일상 vlog 복학생 Ep25,안녕하세요 경돼입니다 요즘 들어 정말 잘 먹고 다녀서 그런지 살이 찌고 있네요 ...,안녕하세요 n과자가 맛있겠네여n저는 과자 잘안먹어여 한달에 한번 먹으면 맛있더라구요...
4,2016-10-19,16,2016,10,19,13,560,2240,32.0,0.0,50.0,스쿼트 어깨 하체 대학생활 중간고사 복학생 운동 vlog,하체랑 어깨 중간고사 대학생활 vlog 복학생 Ep24,중간고사 기간이라 운동을 아주 볼륨있게 하지 못하고 있어요 죄송해요 그래...,후면은 숙련된 빌더들도 상당히 느낌잡기 힘든 부분이에요 너무 조바심 갖지 마시고 일...
